In [1]:
import xarray as xr
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

from dataloader import DataLoaderAR
from AR_model   import AR_model
# write_path = '/uio/lagringshotell/geofag/students/metos/hannasv/ERA5_monthly/'
# read_path  = '/uio/lagringshotell/geofag/students/metos/hannasv/dataERA5_grib/'

base = '/uio/lagringshotell/geofag/students/metos/hannasv/'
base = '/home/hanna/lagrings'
#read_path = '/uio/lagringshotell/geofag/students/metos/hannasv/dataERA5_grib/'
read_folder = 'ERA5_monthly'

In [2]:
folder = os.path.join(base, read_folder)

In [3]:
# DataLoader AR?

In [4]:
start = '2015_03'
stop  = '2015_05'
lat   = 30.0
lon   = 0.0

In [5]:
data_loader = DataLoaderAR(start = start, stop = stop, lat = lat, lon = lon)

Load data .... 


/home/hannasv/anaconda3/envs/sciclouds/lib/python3.6/site-packages/xarray/coding/times.py:207: FutureWarning: the 'box' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'box'
  result = pd.to_timedelta(num_timedeltas, unit=units, box=False)
/home/hannasv/anaconda3/envs/sciclouds/lib/python3.6/site-packages/xarray/coding/times.py:207: FutureWarning: the 'box' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'box'
  result = pd.to_timedelta(num_timedeltas, unit=units, box=False)
/home/hannasv/anaconda3/envs/sciclouds/lib/python3.6/site-packages/xarray/coding/times.py:207: FutureWarning: the 'box' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'box'
  result = pd.to_timedelta(num_timedeltas, unit=units, box=False)
/home/hannasv/anaconda3/envs/sciclouds/lib/python3.6/site-packages/xarray/coding/times.py:207: FutureWarning: the 'box' keyw

In [6]:
data_loader.X.shape

(2208, 5)

In [7]:
data_loader.y.shape

(2208, 1)

In [ ]:
# files = glob.glob(os.path.join(read_path, '*.grib'))

In [ ]:
# domain

In [ ]:
data = xr.open_dataset(files[6], engine = 'cfgrib')
domain = data.sel(latitude = slice(50,30), longitude = slice(-15, 25))

In [ ]:
def dataset_to_numpy(pixel, bias = True):
    """ Dataset. """ 
    n = len(pixel.time.values)
    
    if bias:
        num_vars = 4+1
    X = np.zeros((n, num_vars))
    
    q   = pixel.q.values
    t2m   = pixel.t2m.values
    r   = pixel.r.values
    sp  = pixel.sp.values
    tcc = pixel.tcc.values
        
    X[:, 0] = q
    X[:, 1] = t2m
    X[:, 2] = r
    X[:, 3] = sp
    X[4, :] = 1
    return X, tcc[:, np.newaxis]

In [ ]:
domain

In [ ]:
var = 'r'
months = np.arange(1, 13)
#years = np.arange(2012, 2019)
years=np.arange(2004, 2012)

In [ ]:
for m in months:
    for y in years:
        subset = domain.sel(time = '{}-{:02d}'.format(y, m))
        filename = '{}_{:02d}_{}.nc'.format(y, m, var)
        eng = 'netcdf4'
        encoding_dict = {'{}'.format(var): {'zlib': True, 'complevel': 9}}
        subset.to_netcdf(os.path.join(write_path, filename), encoding=encoding_dict, engine = eng)

# Deal with missing values
## Merge datasets and only keep the times present in both 

In [ ]:
import xarray as xr
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import scipy 

write_path = '/uio/lagringshotell/geofag/students/metos/hannasv/ERA5_monthly/'
read_path = '/uio/lagringshotell/geofag/students/metos/hannasv/dataERA5_grib/'

In [ ]:
results_path = '/uio/lagringshotell/geofag/students/metos/hannasv/results/ar/models/'

In [ ]:
# Store R2, MSE, Weights, 
# Names Wt2m, Wr, Wq, Wsp, W1, W2, .... Wn

In [ ]:
from scipy.linalg import inv

In [ ]:
files = glob.glob(os.path.join(write_path, '2012_05_*.nc'))
data = [xr.open_dataset(fil) for fil in files]
most = xr.merge(data, 'equals')
res = np.zeros((n_lat, n_lon, len(coeffs)))
latitudes = most.latitude.values
longitudes = most.longitude.values
    
n_lat = 81
n_lon = 161

for i, lat in enumerate(latitudes): # 81
    for j, lon in enumerate(longitudes): # 161

        pixel = most.sel(latitude = lat, longitude = lon)
        X, y = dataset_to_numpy(pixel)
        coeffs = np.dot(inv(np.dot(X.T, X)), np.dot(X.T, y))
        res[i, j, :] =  coeffs.flatten()

In [ ]:
temp_dict = {}
order = 3

for i in range(order):
    var = 'W{}'.format(i+1)
    temp_dict[var] = (['latitude', 'longitude'], np.zeros((n_lat, n_lon)))
    
vars_dict = {'b': (['latitude', 'longitude'], res[:, :, 4]),
             'Wt2m':(['latitude', 'longitude'], res[:, :, 1]),
             'Wr': (['latitude', 'longitude'], res[:, :, 2]),
             'Wq':(['latitude', 'longitude'], res[:, :, 0]),
             'Wsp': (['latitude', 'longitude'], res[:, :, 3]),
              }
temp_dict.update(vars_dict)
ds = xr.Dataset(temp_dict,
                 coords={'longitude': (['longitude'], longitudes),
                         'latitude': (['latitude'], latitudes),
                        })

In [ ]:
ds

# Store the results from the regression in nc-files

* 0: q
* 1: t2m
* 2: r
* 3: sp
* 4: bias

In [ ]:
# Inplace returns None
encoding_dict = {}    
for key in temp_dict.keys():
    encoding_dict[key] = {'zlib': True, 'complevel': 9}

In [ ]:
ds.to_netcdf(path = os.path.join(results_path,  'test.nc'),
             engine='netcdf4',
             encoding =encoding_dict)

In [ ]:
# include training periode in attributes and filename

## Read results file and make a prediction.

In [ ]:
results = xr.open_dataset(os.path.join(results_path,  'test.nc'))

In [ ]:
def predict(X, coeffs):
    """Make prediction"""
    return np.dot(X.T, coeffs)

##  Make prediction and calculate R2 and MSE both training and test
### This file contains
* MSE_train
* MSE_test

* R2_train
* R2_test

In [ ]:
results

In [ ]:
files = glob.glob(os.path.join(write_path, '2012_06_*.nc'))
data = [xr.open_dataset(fil) for fil in files]
most = xr.merge(data, 'equals')

In [ ]:
most

In [ ]:
81*161*720

In [ ]:
import seaborn as sns

In [ ]:
temp = most.tcc.values.mean(axis=0)

In [ ]:
temp.shape

In [ ]:
#temp.nanmean()

In [ ]:
sns.heatmap(temp)

In [ ]:
variable = results.variables.keys()

## Compare results to normalized predictions.

In [ ]:
def results_to_numpy(data, weights):
    """ Dataset. 
    Use weights to determine the order of the AR model.
    
    
    data : xr.Dataset
        contains the data you want to make a prediction on 
    
    weights : xr.Dataset
        Contains the weight and bias used to train this model.
        
    """ 
    weights_keys = weights.variables.keys()
    
    order = len(weights_keys) - 5
    
    # Call function which prepares the data to a format suitable to make a prediction. 
    # This involves 
    test_data = prep_data_for_prediction(data, order)
    weights = pred_weights(weights)
    
    
    
    n = len(dataset.time.values)
    
    if bias:
        num_vars = 4+1
        
    X = np.zeros((n, num_vars))
    
    q   = dataset.q.values
    t2m = dataset.t2m.values
    r   = dataset.r.values
    sp  = dataset.sp.values
    tcc = dataset.tcc.values
    
    X[:, 0] = q
    X[:, 1] = t2m
    X[:, 2] = r
    X[:, 3] = sp
    X[:, 4] = 1
    return X, tcc[:, np.newaxis]

In [ ]:
def get_weights(dataset):
    """
    weights: xr.Dataset
        contains the weights and biases
    
    Returns
    --------------
    
    X : weights in matrix for 
        
    explenation : List[str]
        Exlpaines the content in dimension 2.
    
    TODO: 
        Make sure they come in the correct oder
        
    """
    variables = dataset.variables.keys()    
    X = np.zeros((n_lat, n_lon, len(variables)))
    
    W_q   = dataset.Wq.values
    W_t2m = dataset.Wt2m.values
    W_r   = dataset.Wr.values
    W_sp  = dataset.Wsp.values
    b = dataset.b.values
    
    X[:, :, 0] = W_q[:,:]
    X[:, :, 1] = W_t2m[:,:]
    X[:, :, 2] = W_r[:,:]
    X[:, :, 3] = W_sp[:,:]
    X[:, :, 4] = b # bias
    
    if len(variables) > 5+2: # 2 since it contains the latitude, longitude information.
        for i in range(1, len(variables)-2-5+1):
            var = 'W{}'.format(i)
            X[:, :, i+4] = dataset[var].values                      
    return X

# Make predictions 

In [ ]:
def mean_squared_error(y_true, y_pred):
    """Computes the Mean Squared Error score metric."""
    mse = np.square(np.subtract(y_true, y_pred)).mean(axis = 0)
    return mse

def r2_score(y_true, y_pred):
    """  """
    numerator = np.square(np.subtract(y_true, y_pred)).sum()
    denominator = np.square(np.subtract(y_true, np.average(y_true))).sum()
    val = numerator/denominator
    return 1 - val

In [ ]:
mse = mean_squared_error(true, prediction)